In [1]:
from lark import Lark, Transformer

grammar = r"""
  string: ESCAPED_STRING
  %import common.ESCAPED_STRING
"""
parser = Lark(grammar, start='string')

parser.parse('"Hello world"')

def parse(text):
    print(parser.parse(text).pretty())

parse('"Hello world"')

string	"Hello world"



In [2]:
grammar = r"""
   num: SIGNED_NUM
   SIGNED_NUM: /
    [+-]?(0|[1-9][0-9]*[.]?[0-9]*)  # the digits
   /x  
   %ignore " "
"""

parser = Lark(grammar, start='num')
parse('42')

num	42



In [22]:
grammar = r"""
   ?num: start
   ?start: expr|signed_num
   ?expr: add|sub|mul|signed_num
   ?add: br|add "+" sub| sub | br
   ?sub: br|sub "-" mul|mul | br
   ?mul: br|mul "*" div | div | br
   ?div: br|div "/" div | signed_num | br
   ?br: "("expr")"
   signed_num: /[-]?(0|[1-9][0-9]*[.]?[0-9]*)/
   %ignore " "
"""

parser = Lark(grammar, start='num')
parse("9*4*(2+9)*((1+8)/1*2)*5")

mul
  mul
    mul
      mul
        signed_num	9
        signed_num	4
      add
        signed_num	2
        signed_num	9
    mul
      div
        add
          signed_num	1
          signed_num	8
        signed_num	1
      signed_num	2
  signed_num	5



In [21]:
from lark import Transformer

class MyTransformer(Transformer):
    
    def signed_num(self, value):
        (value, ) = value
        return int(value)

    def add(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] + items[1]
    
    def sub(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] - items[1]
        
    def mul(self, items):
        if len(items) == 1: 
            return items[0]
        return items[0] * items[1]
 
    def div(self, items):
        if len(items) == 1: 
            return items[0]
        
        return items[0] / items[1]
 
    

tree = parser.parse("9*4*(2+9)*((1+8)/1*2)*5")
# tree = parser.parse("5/2*9/(1/6-(9-7))")
MyTransformer().transform(tree)

35640.0

In [ ]:
# TEST CODE GENERATED BY CHATGPT (randomly generated nested expression trees with optional brackets based on given seed)
# Generated code was manually modified (see comments)
import random

functions = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x / y,
}

precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
}

# Class to represent a production (recursively used to represent functions)
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value      # operator symbol or number
        self.left = left
        self.right = right

    def is_leaf(self):
        return self.left is None and self.right is None

# Randomly generates an expression tree based on seed
def random_tree(max_depth=3, p_leaf=0.3, seed=1):
    """
    Generate a random expression tree.
    """
    random.seed(seed)
    if max_depth == 0 or random.random() < p_leaf:
        return Node(random.randint(1, 9))

    op = random.choice(list(functions.keys()))
    left = random_tree(max_depth - 1, p_leaf, seed+2)
    right = random_tree(max_depth - 1, p_leaf, seed-33)

    return Node(op, left, right)

# Get the textual representation of the function encoded in the expression tree
def to_string(node, parent_op=None, is_right_child=False, force_brackets=False):
    if node.is_leaf():
        return str(node.value)

    left_str = to_string(node.left, node.value, False, force_brackets)
    right_str = to_string(node.right, node.value, True, force_brackets)

    expr = f"{left_str}{node.value}{right_str}"

    if parent_op is None:
        return expr

    # ======= START: MANUALLY MODIFIED GENERATED CODE ========
    # Parentheses rules
    # Always for lower presendence (eg + and - before / and *)
    if precedence[node.value] < precedence[parent_op] or force_brackets:
        return f"({expr})"

    #
    if (precedence[node.value] == precedence[parent_op]
        and is_right_child):
        return f"({expr})"
    # ======= END: MANUALLY MODIFIED GENERATED CODE ========
    return expr

# Visits expression tree, recursively returns the evaluation of said tree
def evaluate(node):
    if node.is_leaf():
        return node.value

    f = functions[node.value]
    return f(evaluate(node.left), evaluate(node.right))




In [175]:
# TEST CONFIGS
epsilon = 0.00000000001 # ignore floating point errors
SEED = 123456789
NR_TESTS = 500
PRINT_GENERATED_FUNCTIONS = False

# Automatically run tests according to configs. Compares transformed tree result to expression tree.
all_pass = True
for i in range(NR_TESTS):
    r_tree = random_tree(max_depth=4, seed=i*SEED)
    expr = to_string(r_tree)
    expr_exp = to_string(r_tree, force_brackets=True)
    try:
        value = evaluate(r_tree)
    except ZeroDivisionError as err:
        print(f"(i={i}): Generated function \"{expr}\" divides by zero, skipping")
        continue
    tree = parser.parse(expr)
    transformer_value = MyTransformer().transform(tree)
    PASS = (transformer_value - value) < epsilon
    if (not PASS):
        all_pass = False
        print(expr_exp)
        print(f"FAIL (i={i}): Expected {value}, got {transformer_value} for {expr}")
    elif PRINT_GENERATED_FUNCTIONS:
        print(f"(i={i}): {expr}")
if all_pass:
    print("All tests pass!")
else:
    print("FAIL")

(i=62): Generated function "4*(4-2-2)/((4-2-2)/4)" divides by zero, skipping
(i=143): Generated function "(4-3)/(3-3)+(3-3)/(3*3)+3" divides by zero, skipping
(i=286): Generated function "3-9/((6-6)/(6-3))" divides by zero, skipping
(i=399): Generated function "2/7-8+9-9/(7/6/(6-6))" divides by zero, skipping
(i=425): Generated function "(2/8/(8-4)+(8-4)/(4-5))/((8-4)/(4-5)+(4-5+5*1))" divides by zero, skipping
(i=457): Generated function "1*(3+6-4)/((3+6-4)*(4+(2-6)))" divides by zero, skipping
All tests pass!
